# 1️⃣ Setup & Paths

In questa prima cella configuriamo l’ambiente Python:
- Importiamo tutte le librerie necessarie al progetto (PyTorch, Qiskit, pandas, matplotlib, seaborn).
- Aggiungiamo la radice del progetto al `sys.path` in modo da poter importare il codice `qcnn_medmnist` in modalità editable.
- Definiamo i percorsi principali: `DATA_DIR`, `LOGS_DIR` e `REPORTS_DIR`.
- Inizializziamo il backend di matplotlib per l’output inline.
- Verifichiamo la versione delle librerie principali per riproducibilità.




In [ ]:
import sys, os
from pathlib import Path

# 1) Permette di importare il codice in editable mode
PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# 2) Import librerie
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from qcnn_medmnist.utils.seed import set_global_seed

# 3) Seed di riproducibilità globale
SEED = 123
set_global_seed(SEED)

# 4) Qt o inline for matplotlib
%matplotlib inline

# 5) Percorsi principali
DATA_DIR    = PROJECT_ROOT / "data" / "processed"
LOGS_DIR    = PROJECT_ROOT / "logs"
REPORTS_DIR = PROJECT_ROOT / "reports"

# 6) Crea le cartelle se non esistono
LOGS_DIR .mkdir(exist_ok=True)
REPORTS_DIR .mkdir(exist_ok=True)

# 7) Version check
print(f"torch   {torch.__version__}")
print(f"pandas  {pd.__version__}")
print(f"seaborn {sns.__version__}")

[seed] Seme globale impostato a 123
torch   2.7.0+cpu
pandas  2.2.3
seaborn 0.13.2


# 2️⃣ Selezione parametri

In questa cella possiamo **configurare dinamicamente**:
- Il `dataset_name` tra i sottoinsiemi MedMNIST 2D
- Le frazioni `subset`, `subset_val`, `subset_test` per FAST-MODE
- Lo `stride` per la patchificazione
- Il flag `freeze_q` per congelare i parametri quantistici
- Numero di `epochs`, `batch_size` e `device`




In [ ]:
# Scegliamo il dataset (pathmnist, bloodmnist, dermamnist, chestmnist, etc.)
dataset_name = "bloodmnist"

# FAST-MODE: frazione del dataset per train/val/test
subset       = 0.05     # 100%
subset_val   = 0.05     # se None = subset
subset_test  = 0.05     # se None = subset

# Patchify stride
stride       = 3       # deve dividere 30 esattamente

# Congela θ quantistici?
freeze_q     = False

# Addestramento
batch_size   = 32
epochs       = 2
device       = "cpu"    # oppure "cuda"




# 3️⃣ Pre-processing

In questa cella lanciamo il preprocess:
1. Scarica il dataset MedMNIST scelto  
2. Riduce da RGB→grayscale tramite la rete `Color2GrayNet`  
3. Salva i tensori (30×30 con padding) in `data/processed/<dataset>/{train,val,test}.pt`  

Usiamo lo script CLI `qcnn-preprocess` tramite `subprocess`,
che produce progress bar e conferma di salvataggio.



In [3]:
import subprocess

cmd = [
    "qcnn-preprocess",
    f"--dataset={dataset_name}",
    "--epochs=2",
    "--batch=128"
]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)

Running: qcnn-preprocess --dataset=bloodmnist --epochs=2 --batch=128


CompletedProcess(args=['qcnn-preprocess', '--dataset=bloodmnist', '--epochs=2', '--batch=128'], returncode=0)



# 4️⃣ Training

Qui costruiamo e lanciamo il training in‐code:
- Importiamo la classe `Trainer` dal modulo Python
- Calcoliamo automaticamente la `run_id` (001, 002, …)
- Istanziamo `Trainer(...)` con tutti i parametri
- Lanciamo `fit()` con barra di progresso  
- Alla fine lanciamo `test()` e vediamo le metriche finali.




In [4]:
from qcnn_medmnist.scripts.train import _next_run_id
from qcnn_medmnist.training.trainer import Trainer

# 1) Determina la cartella di run
run_id = _next_run_id(dataset_name)
out_dir = LOGS_DIR / f"{dataset_name}_run_{run_id:03d}"

# 2) Istanzia e addestra
trainer = Trainer(
    dataset_name= dataset_name,
    out_dir      = out_dir,
    batch_size   = batch_size,
    lr           = 1e-3,
    epochs       = epochs,
    device       = device,
    subset       = subset,
    subset_val   = subset_val,
    subset_test  = subset_test,
    stride       = stride,
    freeze_q     = freeze_q,
)
trainer.fit()

# 3) Test finale e metriche
test_metrics = trainer.test()
print("Test metrics:", test_metrics)

15:05:18 | WARNING | FAST-MODE  train subset=0.05  →  508 campioni
15:05:18 | WARNING | FAST-MODE  val subset=0.05  →  89 campioni
15:05:18 | WARNING | FAST-MODE  test subset=0.05  →  171 campioni
15:05:18 | INFO | Numero classi rilevate: 8
15:05:20 | INFO | ▶️  Inizio training (2 epoche)…


15:14:59 | INFO | [Ep 1/2] train_loss=2.1382  val_loss=2.0312  val_acc=0.213


15:14:59 | SUCCESS |   ↳  Nuovo best model salvato  (val_loss 2.0312)


15:25:24 | INFO | [Ep 2/2] train_loss=2.0001  val_loss=1.9257  val_acc=0.292
15:25:24 | SUCCESS |   ↳  Nuovo best model salvato  (val_loss 1.9257)


15:25:24 | SUCCESS | ✅ Training completato!


Test metrics: {'acc': 0.22807017543859648, 'bal_acc': np.float64(0.14186401810759963), 'f1': 0.10691593352883674, 'auc_micro': nan, 'auc_macro': nan, 'loss': 1.98112162023957}
15:28:25 | INFO | ��  Test  acc=0.228  f1=0.107


# 5️⃣ Report & Plot

Ora generiamo:
1. CSV delle predizioni (`reports/<run>/tables/preds.csv`)
2. Confusion‐matrix inline e salva PNG
3. Curve di learning inline e salva PNG



In [ ]:
import matplotlib.pyplot as plt
from qcnn_medmnist.scripts.report import generate_report
from qcnn_medmnist.scripts.plot_curves import generate_learning_curves

# A) Chiama la funzione di report (salva preds.csv + cm + meta)

run_dir = LOGS_DIR / f"{dataset_name}_run_{run_id:03d}"
generate_report(dataset_name, run_dir, stride)

# B) Learning curves (salva i PNG)

generate_learning_curves(run_dir)

# C) Ora mostriamo **tutte** le immagini salvate in reports/.../figures

figs_dir = REPORTS_DIR / run_dir.name / "figures"
for img_path in sorted(figs_dir.glob("*.png")):
    img = plt.imread(img_path)
    plt.figure(figsize=(5,5))
    plt.imshow(img, aspect="equal")
    plt.axis("off")
    plt.title(img_path.name, fontsize=10)
    plt.show()

# 6️⃣ Heat-map interattiva

Visualizziamo inline la heat‐map ⟨Z⟩ per una immagine di test:
- Scegli `idx` dell’immagine (da 0 a N_test–1)
- Calcola patch e attivazioni via `model.qconv`
- Traccia una heatmap seaborn in‐line
- Salva in `reports/<run>/figures/heatmap_idx<idx>.png`


In [ ]:
from qcnn_medmnist.scripts.heatmap import generate_heatmap

# Genera e salva la heat‐map per l’indice selezionato
idx = 0
generate_heatmap(dataset_name, run_dir, idx=idx, stride=stride)

# Carica e mostra inline con plt.imshow()
heat_path = REPORTS_DIR / run_dir.name / "figures" / f"heatmap_idx{idx}.png"
img = plt.imread(heat_path)
plt.figure(figsize=(5,5))
plt.imshow(img, cmap=None)
plt.axis("off")
plt.title(f"Heat‐map idx={idx}", fontsize=12)
plt.show()